# Input Classification and Routing

## Overview

Production LLM systems must **classify and route** user inputs to appropriate handlers. This notebook demonstrates:

1. **Intent Classification** - Identifying the user's goal from free-form text
2. **Hierarchical Categorization** - Primary and secondary category assignment
3. **Structured Output** - JSON format for downstream processing
4. **Delimiter-Based Isolation** - Preventing prompt injection attacks

## Why Input Classification Matters

### Production Use Cases
- **Customer Support** - Route tickets to specialized teams (billing, technical, account)
- **Chatbots** - Trigger specific workflows based on intent
- **Content Moderation** - Flag queries requiring human review
- **Analytics** - Track query distribution and user behavior

### Benefits
1. **Cost Optimization** - Route simple queries to cheaper models/handlers
2. **Response Quality** - Specialized handlers for each category
3. **Scalability** - Distribute load across services
4. **Compliance** - Ensure sensitive queries go to certified systems

## Key Concepts

### Delimiters for Safety
Using special characters (e.g., `####`) to wrap user input prevents:
- **Prompt Injection** - User can't override system instructions
- **Parsing Errors** - Clear boundary between system and user content

### JSON Output
Structured format enables:
- Programmatic routing logic
- Database storage
- Analytics and reporting

## Environment Setup

In [ ]:
import os
import openai
import tiktoken
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

---

## Hierarchical Classification System

This system uses **two-level taxonomy**:
- **Primary Category**: High-level intent (Billing, Technical Support, Account Management, General Inquiry)
- **Secondary Category**: Specific action within primary category

### Architecture Pattern

```
User Input → Delimiter Wrap → LLM Classifier → JSON Output → Route to Handler
```

### System Message Design

The system message provides:
1. **Task Definition** - What to do with the input
2. **Category Taxonomy** - Complete list of valid categories
3. **Output Format** - JSON structure specification
4. **Delimiter Instructions** - How input is wrapped

**Pro Tip**: Comprehensive category lists improve classification accuracy.

In [ ]:
client = openai.OpenAI()
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, 
                                 max_tokens=500):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        max_tokens=max_tokens, # the maximum number of tokens the model can ouptut 
    )
    return response.choices[0].message.content

**Expected Output**:
```json
{
  "primary": "Account Management",
  "secondary": "Close account"
}
```

**Routing Logic**: This JSON can programmatically trigger account deletion workflow.

---

## Edge Case: Ambiguous or Frustrated Input

Testing classification when user intent is unclear or expressed with frustration.

In [ ]:
delimiter = "####"
system_message = f"""
You will be provided with customer service queries. \
The customer service query will be delimited with \
{delimiter} characters.
Classify each query into a primary category \
and a secondary category. 
Provide your output in json format with the \
keys: primary and secondary.

Primary categories: Billing, Technical Support, \
Account Management, or General Inquiry.

Billing secondary categories:
Unsubscribe or upgrade
Add a payment method
Explanation for charge
Dispute a charge

Technical Support secondary categories:
General troubleshooting
Device compatibility
Software updates

Account Management secondary categories:
Password reset
Update personal information
Close account
Account security

General Inquiry secondary categories:
Product information
Pricing
Feedback
Speak to a human

"""
user_message = f"""\
I want you to delete my profile and all of my user data"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

**Expected Output**:
```json
{
  "primary": "General Inquiry",
  "secondary": "Speak to a human"
}
```

**Routing Logic**: Escalate to human agent when intent is unclear or user is frustrated.

---

## Production Implementation Patterns

### 1. Routing Logic (Pseudocode)

```python
import json

classification = json.loads(response)

if classification["primary"] == "Billing":
    route_to_billing_team(user_message)
elif classification["primary"] == "Technical Support":
    route_to_tech_support(user_message)
elif classification["secondary"] == "Speak to a human":
    escalate_to_human_agent(user_message)
else:
    route_to_general_chatbot(user_message)
```

### 2. Confidence Scoring

For production, add confidence scores:
```python
system_message += """
Also provide a confidence score (0-1) for your classification.
Output format: {"primary": "X", "secondary": "Y", "confidence": 0.95}
"""
```

Route low-confidence queries to human review.

### 3. Multi-Label Classification

Some queries span multiple categories:
```python
user_message = "I want to cancel my subscription and update my email"
# Should classify as both "Billing" (Unsubscribe) and "Account Management" (Update info)
```

Modify system message to support arrays:
```json
{
  "categories": [
    {"primary": "Billing", "secondary": "Unsubscribe or upgrade"},
    {"primary": "Account Management", "secondary": "Update personal information"}
  ]
}
```

### 4. Fallback Handling

Always handle edge cases:
- Invalid JSON response → Log error, route to human
- Unknown category → Route to general inquiry handler
- Empty input → Prompt user for clarification

---

## Summary: Classification Best Practices

### System Message Design
1. **Explicit Taxonomy** - List all valid categories (prevents hallucination)
2. **Output Format** - Specify JSON structure clearly
3. **Edge Cases** - Include "Other" or "General Inquiry" catch-all
4. **Examples** - Optionally add few-shot examples for complex cases

### Security
1. **Delimiters** - Always wrap user input to prevent injection
2. **Validation** - Parse JSON and validate against schema
3. **Sanitization** - Strip dangerous content before processing

### Performance
1. **Caching** - Cache classifications for repeated queries
2. **Batch Processing** - Classify multiple inputs in one API call
3. **Model Selection** - Use cheaper models (gpt-3.5-turbo) for classification

### Monitoring
1. **Track Distribution** - Monitor category frequency
2. **Audit Low Confidence** - Review borderline classifications
3. **User Feedback** - Allow users to correct misclassifications

### Next Steps
- **C.InputModerationTechniques** - Content safety before classification
- **D.ChainOfThoughtReasoning** - Complex reasoning for classification
- **E.PromptChaining** - Multi-step classification pipelines

In [ ]:

user_message = f"""\
I need urgent help, idk what  you want or are saying"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 

response = get_completion_from_messages(messages)
print(response)